<a href="https://colab.research.google.com/github/ecribbie/6912_PROJECT/blob/main/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
!git clone https://github.com/ecribbie/6912_PROJECT.git
%cd ./6912_PROJECT/

Cloning into '6912_PROJECT'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 15), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (49/49), 1.38 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/content/6912_PROJECT/6912_PROJECT/6912_PROJECT


## Read data

In [15]:
!ls

 data_analysis.ipynb  'default of credit card clients.xls'   README.md


In [16]:
data=pd.read_excel("default of credit card clients.xls")
data.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [17]:
import csv

# Data to be written to the CSV file
data = [
    ['Name', 'Age', 'City'],
    ['John', 30, 'New York'],
    ['Alice', 25, 'Los Angeles'],
    ['Bob', 35, 'Chicago']
]

# Define the file path
file_path = 'test.csv'

# Open the CSV file in write mode
with open(file_path, 'w', newline='') as file:
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the data to the CSV file
    writer.writerows(data)

In [27]:
!git config --global user.name "Evan"
!git config --global user.email "ecribbie@sfu.ca"

In [23]:
!git add .
!git commit -m "update"
!git push

[main 783ca1b] update
 1 file changed, 4 insertions(+)
 create mode 100644 test.csv
fatal: could not read Username for 'https://github.com': No such device or address


In [28]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
